In [6]:
!pip install ydata-synthetic


In [7]:
!pip install faker

In [8]:
import random
import pandas as pd
import faker

fake = faker.Faker()

# Configuration
NUM_AUTHORS = 100
NUM_SUBJECTS = 50
NUM_AFFILIATIONS = 20
NUM_ARTICLES = 200

# Generate authors
authors = [fake.name() for _ in range(NUM_AUTHORS)]

# Generate subjects
subjects = [fake.word() for _ in range(NUM_SUBJECTS)]

# Generate affiliations
affiliations = [fake.company() for _ in range(NUM_AFFILIATIONS)]

# Generate articles
articles = []
for _ in range(NUM_ARTICLES):
    article = {
        'title': fake.sentence(nb_words=6),
        'author': random.choice(authors),
        'subject': random.choice(subjects),
        'affiliation': random.choice(affiliations),
        'publication_year': random.randint(2000, 2024)
    }
    articles.append(article)

# Convert to DataFrame
articles_df = pd.DataFrame(articles)

print(articles_df.head())


                                            title               author  \
0                     Site quality pattern night.          Sara Glover   
1  Remember her easy she peace fight head street.  Michelle Jacobs PhD   
2         Almost development door concern moment.     Johnathan Zuniga   
3           Enough off join last that tax animal.      Robert Thompson   
4                                Mind price both.       Rachael Miller   

      subject           affiliation  publication_year  
0       since       Mcdaniel-Vargas              2009  
1       radio          Frost-Wilson              2006  
2  successful          Parks-Powers              2011  
3        just  Smith, Gray and Gray              2020  
4    possible          Dodson Group              2005  


In [9]:
import pandas as pd
import random

# Define realistic subjects
subjects = [
    "Teknik Informatika", "Teknik Elektro", "Teknik Mesin", "Fisika",
    "Matematika", "Biologi", "Kimia", "Psikologi", "Sosiologi", "Ekonomi",
    "Ilmu Politik", "Sejarah", "Filsafat", "Sastra", "Linguistik",
    "Seni", "Musik", "Ilmu Lingkungan", "Kedokteran", "Hukum"
]

# Define realistic affiliations
affiliations = [
    "Universitas Indonesia", "Institut Teknologi Bandung", "Universitas Gadjah Mada",
    "Institut Pertanian Bogor", "Universitas Airlangga", "Universitas Padjadjaran",
    "Institut Teknologi Sepuluh Nopember", "Universitas Diponegoro",
    "Universitas Hasanuddin", "Universitas Brawijaya", "Universitas Sebelas Maret",
    "Universitas Sumatera Utara", "Universitas Pendidikan Indonesia",
    "Universitas Andalas", "Universitas Negeri Malang",
    "Universitas Udayana", "Universitas Jenderal Soedirman",
    "Universitas Syiah Kuala", "Universitas Lampung", "Universitas Sam Ratulangi"
]

# Generate articles with discrete data
articles = []
NUM_ARTICLES = 200

for _ in range(NUM_ARTICLES):
    article = {
        'title': fake.sentence(nb_words=6),
        'author': fake.name(),
        'subject': random.choice(subjects),
        'affiliation': random.choice(affiliations),
        'publication_year': random.randint(2000, 2024)
    }
    articles.append(article)

# Convert to DataFrame
articles_df = pd.DataFrame(articles)

print(articles_df.head())

articles_df['title'].head()

                                           title         author  \
0                           Follow another five.   Laura Miller   
1                        Prevent amount mission.  Haley Douglas   
2   Across nothing meeting build contain expect.   Jerry Thomas   
3                     Grow call blood win focus.  Miguel Tucker   
4  Big charge maintain will security film point.    Mason Barry   

              subject                          affiliation  publication_year  
0             Sejarah                Universitas Indonesia              2017  
1              Fisika                Universitas Brawijaya              2019  
2             Ekonomi  Institut Teknologi Sepuluh Nopember              2010  
3  Teknik Informatika       Universitas Jenderal Soedirman              2011  
4  Teknik Informatika            Universitas Sam Ratulangi              2012  


,title
0,Follow another five.
1,Prevent amount mission.
2,Across nothing meeting build contain expect.
3,Grow call blood win focus.
4,Big charge maintain will security film point.


In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(stop_words='english')
articles_df['title'] = articles_df['title'].fillna('')  # Fill NaNs with empty strings
tfidf_matrix = tfidf.fit_transform(articles_df['title'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the article that matches the title
    idx = articles_df[articles_df['title'] == title].index[0]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles
    sim_scores = sim_scores[1:11]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar articles
    return articles_df.iloc[article_indices]

recommendations = get_recommendations(articles_df['title'].iloc[0:1].to_string(index=False))
recommendations[['title', 'author']]


,title,author
13,Speak young grow debate follow behind.,Lisa Valdez
1,Prevent amount mission.,Haley Douglas
2,Across nothing meeting build contain expect.,Jerry Thomas
3,Grow call blood win focus.,Miguel Tucker
4,Big charge maintain will security film point.,Mason Barry
5,Sister finish service energy respond visit gar...,Dr. Emily Wilson MD
6,Second worry social someone different join din...,Lisa Flores
7,Party tend maybe factor mean type.,Dale Valenzuela
8,Animal debate hot American happen attention.,Spencer Jefferson
9,Although site several deep generation anything...,Tara Snyder


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Split data into training and testing sets
train_df, test_df = train_test_split(articles_df, test_size=0.2, random_state=42)

# Function to get recommendations based on title
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
        idx = articles_df[articles_df['title'] == title].index[0]
    except IndexError:
        return []

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]

    return articles_df.iloc[article_indices]

# Create a function to evaluate our recommender system
def evaluate_recommender(train_df, test_df, tfidf, cosine_sim):
    y_true = []
    y_pred = []

    for index, row in test_df.iterrows():
        recommendations = get_recommendations(row['title'], cosine_sim)

        if len(recommendations) == 0:
            continue

        true_subjects = [row['subject']] * len(recommendations)
        pred_subjects = recommendations['subject'].tolist()

        y_true.extend(true_subjects)
        y_pred.extend(pred_subjects)

    if len(y_true) == 0 or len(y_pred) == 0:
        return 0, 0, 0  # Avoid division by zero if there's no valid prediction

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    return precision, recall, f1

# Example usage
precision, recall, f1 = evaluate_recommender(train_df, test_df, tfidf, cosine_sim)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")


In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Combine title, subject, and affiliation into a single string
articles_df['content'] = articles_df['title'] + " " + articles_df['subject'] + " " + articles_df['affiliation']

# Vectorize the 'content' field to create TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(articles_df['content'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get recommendations based on content
def get_recommendations(content, cosine_sim=cosine_sim):
    try:
        idx = articles_df[articles_df['content'] == content].index[0]
    except IndexError:
        return []

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]

    return articles_df.iloc[article_indices]

# Example usage
print(get_recommendations(articles_df['content'].iloc[0]))

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Split data into training and testing sets
train_df, test_df = train_test_split(articles_df, test_size=0.2, random_state=42)

# Create a function to evaluate our recommender system
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the article that matches the title
    idx = articles_df[articles_df['title'] == title].index[0]

    # Get the pairwise similarity scores of all articles with that article
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the articles based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar articles
    sim_scores = sim_scores[1:11]

    # Get the article indices
    article_indices = [i[0] for i in sim_scores]

    # Create a DataFrame for the recommendations
    recommendations = articles_df.iloc[article_indices].copy()

    # Add similarity scores to the recommendations DataFrame
    recommendations['similarity_score'] = [i[1] for i in sim_scores]

    # Add textual explanations
    recommendations['explanation'] = recommendations.apply(
        lambda row: f"The similarity score is {row['similarity_score']:.2f}. ",
        axis=1
    )

    return recommendations

recommendations = get_recommendations(articles_df['title'].iloc[0:1].to_string(index=False))
recommendations[['title', 'subject', 'explanation']]


# # Example usage
# precision, recall, f1 = evaluate_recommender(train_df, test_df, cosine_sim)
# print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")


                                                title               author       subject                       affiliation  publication_year                                            content
13             Speak young grow debate follow behind.          Lisa Valdez         Kimia             Universitas Indonesia              2010  Speak young grow debate follow behind. Kimia U...
136                        Body then four house onto.          Carla Mason       Sejarah             Universitas Airlangga              2021  Body then four house onto. Sejarah Universitas...
10             History interesting because few radio.            Jerry Lee       Sejarah            Universitas Hasanuddin              2024  History interesting because few radio. Sejarah...
22                          Tough product wonder the.          Lori Carter       Sejarah        Universitas Sumatera Utara              2003  Tough product wonder the. Sejarah Universitas ...
159     Particularly before drug even th

,title,subject,explanation
13,Speak young grow debate follow behind.,Kimia,The similarity score is 0.40.
136,Body then four house onto.,Sejarah,The similarity score is 0.26.
10,History interesting because few radio.,Sejarah,The similarity score is 0.23.
22,Tough product wonder the.,Sejarah,The similarity score is 0.22.
159,Particularly before drug even thought budget.,Sejarah,The similarity score is 0.20.
52,Note technology cost instead group.,Sejarah,The similarity score is 0.18.
74,Similar region drive according.,Sejarah,The similarity score is 0.17.
18,Boy over piece thing support movie end behavior.,Sejarah,The similarity score is 0.16.
126,Employee about truth.,Psikologi,The similarity score is 0.16.
138,Your particularly question nation.,Ilmu Politik,The similarity score is 0.14.


In [18]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             200 non-null    object
 1   author            200 non-null    object
 2   subject           200 non-null    object
 3   affiliation       200 non-null    object
 4   publication_year  200 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 7.9+ KB


In [20]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

articles_df.head()

,title,author,subject,affiliation,publication_year
0,Follow another five.,Laura Miller,Sejarah,Universitas Indonesia,2017
1,Prevent amount mission.,Haley Douglas,Fisika,Universitas Brawijaya,2019
2,Across nothing meeting build contain expect.,Jerry Thomas,Ekonomi,Institut Teknologi Sepuluh Nopember,2010
3,Grow call blood win focus.,Miguel Tucker,Teknik Informatika,Universitas Jenderal Soedirman,2011
4,Big charge maintain will security film point.,Mason Barry,Teknik Informatika,Universitas Sam Ratulangi,2012


In [26]:
articles_df['title'].iloc[0:1].to_string(index=False)

'Follow another five.'